In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
from nltk.corpus import stopwords
import spacy
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Embedding, LSTM,GRU
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os
import shutil
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df=pd.read_csv(r"C:\Users\dkdes\OneDrive\Desktop\kaggle_datasets\cleaned_real_estate_data.csv")
df.head()

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,Emirate,location,price_per_sqft,has_plus_bedroom,has_plus_bathroom,Total_rooms
0,Great ROI I High Floor I Creek View,"binghatti canal, business bay, dubai",3.0,2.0,2024-08-14T12:02:53Z,Residential for Sale,2500000,True,sell,1323.0,NO,MNA Properties is delighted to offer this apar...,dubai,business bay,1889.644747,False,False,5.0
1,Full Sea View | Beach Life | Brand New Residence,"la vie, jumeirah beach residence, dubai",3.0,2.0,2024-08-13T05:45:16Z,Residential for Sale,7200000,True,sell,1410.0,YES,"Apartment for sale in La Vie, Jumeirah Beach R...",dubai,jumeirah,5106.382979,False,False,5.0
2,Green Belt | Corner Unit | Spacious Plot,"la rosa 6, villanova, dubai land, dubai",3.0,4.0,2024-08-14T06:24:28Z,Residential for Sale,3600000,True,sell,2324.0,NO,Treo Homes is very pleased to be bringing to t...,dubai,dubai land,1549.053356,False,False,7.0
3,2BR+Study | Near Pool and Park | Private,"springs 15, the springs, dubai",3.0,2.0,2024-08-15T06:07:22Z,Residential for Sale,2999999,True,sell,1647.0,NO,2 Bedrooms + Study | Near Pool &amp; Park | Pr...,dubai,the springs,1821.493018,False,False,5.0
4,Vacant | Well Maintained | Area Expert,"noor townhouses, town square, dubai",3.0,3.0,2024-08-09T08:28:59Z,Residential for Sale,2449999,True,sell,2105.0,NO,-Type 1\n-3 Bed+Maid\n-Close To Amenities\n-BU...,dubai,town square,1163.895012,False,False,6.0


In [4]:
df["displayAddress"][0]

'binghatti canal, business bay, dubai'

In [5]:
df["description"][0]

'MNA Properties is delighted to offer this apartment located in the well-esteemed Binghatti Canal, situated in the heart of Business Bay, Dubai, UAE. The 2BR apartment with the most attractive layout based on the high floor and offers the breathtaking panoramic view of Business Bay Canal; near Burj Khalifa.\n\nTo arrange a viewing or to find out more, please contact Amir via phone or Whatsapp on +971581684981 or email at amir.m@mnaproperties.ae\n\nProperty Features:\n2 Bedrooms\n2 Bathrooms + Guest room\nSize 1,323 sq. ft.\n2 Balconies\nPrivate Jacuzzi / Small Pool\nCanal View\n\nCan be sold Unfurnished as well\n\nBinghatti Canal Residences is a newly built project in Business Bay, Dubai that features symbolic design elements to generate a sense of community. Binghatti Canal reflects Binghatti Developers traditional Middle Eastern architecture and art, in addition to its modern design. Bingatti Canal finishing demonstrate the highest degree of quality and workmanship available only fro

In [6]:
df["description"][1]

'Apartment for sale in La Vie, Jumeirah Beach Residence\n\nMultiple Options\n\nLuxury 2-Bedroom apartment in La Vie, Jumeirah Beach Residences\n\nThis 2-bedroom apartment in La Vie offers a luxurious waterfront living experience with world-class amenities and unparalleled views.\n`Its Large Balcony overlooking Breathtaking views of Sea, the Palm Jumeirah.\n\nLocation: Prime spot in Jumeirah Beach Residences, Easy in and out.\nFeatures:\n\n-2 Bedroom\n-3 Baths\n-Sea Views and Beautiful Sunsets\n-High floor corner unit with a Large balcony\n-Master bedroom with dressing room\n-1 parking space\n- Price Beach Access\n\nFacilities and Amenities\nPrivate Beach\nVIP drop Off\n2 Swimming Pools (Kids &amp; Adults) Pool size - 31 Meter\nVolley-Ball Court\nBeach Club Drop Off\nClub House Pool\nModern huge GYM\nHigh ceiling lobby\nMultipurpose hall\nBespoke Outdoor Gym\nShaded Kids Area\nSteam &amp; Sauna Area (2)\nBBQ Area\nGreen Grass Sitting Area\nHigh speed Elevators\n\nConnectivity:\n-Direct 

In [14]:
string.punctuation
exclude = string.punctuation

In [15]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)
def remove_punc1(text):
    return text.translate(str.maketrans('', '', exclude))
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)
    

In [16]:
df["description"]=df["description"].str.lower()

In [17]:
df["description"]=df["description"].apply(remove_html_tags)
df["description"]=df["description"].apply(remove_url)
df["description"]=df["description"].apply(remove_punc1)
df["description"]=df["description"].apply(remove_stopwords)

In [18]:
df["description"][0]

'mna properties  delighted  offer  apartment located   wellesteemed binghatti canal situated   heart  business bay dubai uae  2br apartment    attractive layout based   high floor  offers  breathtaking panoramic view  business bay canal near burj khalifa  arrange  viewing   find   please contact amir via phone  whatsapp  971581684981  email  amirmmnapropertiesae property features 2 bedrooms 2 bathrooms guest room size 1323 sq ft 2 balconies private jacuzzi small pool canal view   sold unfurnished  well binghatti canal residences   newly built project  business bay dubai  features symbolic design elements  generate  sense  community binghatti canal reflects binghatti developers traditional middle eastern architecture  art  addition   modern design bingatti canal finishing demonstrate  highest degree  quality  workmanship available     reputable worldwide vendors  interior design  fittings  fixtures   2br apartment   perfect harmony providing   excellent balance  interior  outside spaces

In [2]:
from sentence_transformers import SentenceTransformer

C:\Users\dkdes\AppData\Roaming\Python\Python310\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# model=SentenceTransformer("all-MiniLM-L6-V2")

# df["Embeddings"]=df["description"].apply(lambda x:model.encode(x))

NameError: name 'df' is not defined

In [21]:
df["Embeddings"][0]

array([ 9.71012935e-02, -5.09512462e-02,  2.60031484e-02,  2.06886493e-02,
       -8.49905908e-02,  4.65113856e-02, -2.72871889e-02, -4.44946028e-02,
       -6.46912456e-02, -4.59335521e-02,  2.08179355e-02, -4.85441163e-02,
        6.82901070e-02, -3.82898226e-02,  7.80341625e-02,  7.28558470e-03,
        8.71893317e-02,  1.19676320e-02,  2.80943401e-02, -5.73307090e-02,
        3.99382785e-02,  5.62079065e-03,  9.87043604e-03, -8.89688730e-02,
       -4.65664193e-02,  6.63284510e-02,  2.53951866e-02,  6.02790415e-02,
       -2.89485045e-03, -6.66851923e-02,  1.77986249e-02,  1.44315213e-01,
        8.89698789e-02, -2.97663901e-02,  1.18538536e-01,  1.25000447e-01,
       -2.84707043e-02, -3.91993560e-02,  4.70859148e-02, -4.24557738e-02,
       -1.19841844e-02, -4.68790270e-02,  3.07127722e-02, -3.66231874e-02,
        1.54234124e-02, -6.44053668e-02, -2.96297111e-02, -3.07938587e-02,
        1.15674011e-01,  2.80966647e-02, -4.89428081e-02,  2.42914092e-02,
        3.30258086e-02,  

In [8]:
# df.to_pickle("estate_embeddings.pkl")
df=pd.read_pickle(r"C:\Users\dkdes\OneDrive\Desktop\kaggle_datasets\estate_embeddings.pkl")

In [9]:
df.head()

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,Emirate,location,price_per_sqft,has_plus_bedroom,has_plus_bathroom,Total_rooms,Embeddings
0,Great ROI I High Floor I Creek View,"binghatti canal, business bay, dubai",3.0,2.0,2024-08-14T12:02:53Z,Residential for Sale,2500000,True,sell,1323.0,NO,mna properties delighted offer apartment lo...,dubai,business bay,1889.644747,False,False,5.0,"[0.09710129, -0.050951246, 0.026003148, 0.0206..."
1,Full Sea View | Beach Life | Brand New Residence,"la vie, jumeirah beach residence, dubai",3.0,2.0,2024-08-13T05:45:16Z,Residential for Sale,7200000,True,sell,1410.0,YES,apartment sale la vie jumeirah beach residen...,dubai,jumeirah,5106.382979,False,False,5.0,"[0.0934509, -0.047142066, 0.04818736, 0.011824..."
2,Green Belt | Corner Unit | Spacious Plot,"la rosa 6, villanova, dubai land, dubai",3.0,4.0,2024-08-14T06:24:28Z,Residential for Sale,3600000,True,sell,2324.0,NO,treo homes pleased bringing market late...,dubai,dubai land,1549.053356,False,False,7.0,"[0.09101811, -0.090362936, -0.014454419, 0.037..."
3,2BR+Study | Near Pool and Park | Private,"springs 15, the springs, dubai",3.0,2.0,2024-08-15T06:07:22Z,Residential for Sale,2999999,True,sell,1647.0,NO,2 bedrooms study near pool amp park private bu...,dubai,the springs,1821.493018,False,False,5.0,"[0.11712934, -0.005692215, 0.019161439, 0.0334..."
4,Vacant | Well Maintained | Area Expert,"noor townhouses, town square, dubai",3.0,3.0,2024-08-09T08:28:59Z,Residential for Sale,2449999,True,sell,2105.0,NO,type 1 3 bedmaid close amenities bua 2120 sqf...,dubai,town square,1163.895012,False,False,6.0,"[0.14771235, -0.07886978, 0.043340392, 0.07339..."


In [10]:
model=SentenceTransformer("all-MiniLM-L6-V2")

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_estate(query , top_k=5):
    query_embedding=model.encode(query)
    df["similarity"] = df["Embeddings"].apply(lambda x: cosine_similarity([query_embedding], [np.array(x)])[0][0])


    recommendations = df.sort_values(by="similarity" , ascending=False).head(top_k)
    return recommendations[["price","bathrooms","bedrooms","Emirate","location","description","furnishing"]]

    

In [15]:
query = "Apartement near airport"
recommendations = recommend_estate(query)
recommendations

,price,bathrooms,bedrooms,Emirate,location,description,furnishing
1911,33500000,5.0,4.0,dubai,downtown dubai,il primo developed emaar 77storey building ...,NO
509,610000,1.0,0.0,dubai,dubai south,close largest airport world dwc al maktoum ...,NO
1582,2999999,2.0,1.0,dubai,dubai harbour,achiever properties llc proudly presents newl...,YES
2895,3017000,3.0,2.0,sharjah,aljada,near sharja airport 4 main roads sharjah 5 ...,PARTLY
3457,750000,1.0,0.0,dubai,mohammed bin rashid city,mag city meticulously designed based concep...,YES


In [16]:
recommendations["description"].str.contains("airport")

1911    True
509     True
1582    True
2895    True
3457    True
Name: description, dtype: bool

In [14]:
df["Embeddings"]

0       [0.09710129, -0.050951246, 0.026003148, 0.0206...
1       [0.0934509, -0.047142066, 0.04818736, 0.011824...
2       [0.09101811, -0.090362936, -0.014454419, 0.037...
3       [0.11712934, -0.005692215, 0.019161439, 0.0334...
4       [0.14771235, -0.07886978, 0.043340392, 0.07339...
                              ...                        
4691    [0.067871444, -0.010987492, -0.02185905, 0.060...
4692    [0.043114364, 0.011016979, -0.024703698, 0.020...
4693    [0.061551835, -0.08241684, -0.0026803408, 0.03...
4694    [0.009550633, -0.03936216, 0.032077085, 0.0458...
4695    [0.08769903, -0.048356567, -0.028862823, 0.053...
Name: Embeddings, Length: 4696, dtype: object

In [17]:
core_building_amenities = [
    "pool",
    "gymnasium",
    "gym"
    "24/7 security",
    "security",
    "concierge service",
    "parking",
    "high-speed elevator",
    "smart access system"
]

lifestyle_community_amenities = [
    "landscaped gardens",
    "children’s play area",
    "jogging track",
    "cycling track",
    "pet-friendly zone",
    "clubhouse",
    "spa room",
    "sauna",
    "jacuzzi",
    "steam room"
]

location_linked_perks = [
    "proximity to mall",
    "proximity to metro station",
    "waterfront view",
    "burj khalifa view",
    "creek view",
    "near school",
    "near nursery",
    "retail access"
]

investor_focused_addons = [
    "property management service",
    "low service charges",
    "high rental yield zone",
    "developer reputation",
    "tourist-friendly location"
]

In [18]:
core_building_amenities[0]

'swimming pool'

In [25]:
df.loc[df["description"].str.contains(r"\bgym\b")]

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,Emirate,location,price_per_sqft,has_plus_bedroom,has_plus_bathroom,Total_rooms,Embeddings,similarity
1,Full Sea View | Beach Life | Brand New Residence,"la vie, jumeirah beach residence, dubai",3.0,2.0,2024-08-13T05:45:16Z,Residential for Sale,7200000,True,sell,1410.000,YES,apartment sale la vie jumeirah beach residen...,dubai,jumeirah,5106.382979,False,False,5.0,"[0.0934509, -0.047142066, 0.04818736, 0.011824...",0.192513
5,Duplex | Penthouse | Best Downtown Views,"the residences 1, the residences, downtown dub...",7.0,5.0,2024-08-07T07:26:56Z,Residential for Sale,25000000,True,sell,5560.875,YES,duplex penthouse best downtown views penthouse...,dubai,downtown dubai,3541.578127,False,False,12.0,"[0.08974376, -0.07112898, 0.0015430254, 0.0503...",0.166801
6,High Floor | Motivated Seller | Full Marina View,"vida residences dubai marina, dubai marina, dubai",3.0,2.0,2024-08-16T05:03:07Z,Residential for Sale,4980000,True,sell,1095.000,NO,property features 2 bedrooms unfurnished high ...,dubai,dubai marina,4547.945205,False,False,5.0,"[0.05768374, -0.017169476, 0.022535445, 0.0257...",0.154097
13,Sea View | Move In Ready | Great Location,"princess tower, dubai marina, dubai",3.0,2.0,2024-08-07T10:32:26Z,Residential for Sale,2100000,True,sell,1188.000,YES,gta properties proud present 2bed 3 bathroo...,dubai,dubai marina,1767.676768,False,False,5.0,"[0.06770286, -0.09099732, 0.026669133, 0.01027...",0.191204
14,Payment Plan | Close To Amenities | Skylight,"bliss, arabian ranches 3, dubai",4.0,3.0,2024-08-12T11:48:09Z,Residential for Sale,2700000,True,sell,2176.000,NO,sisu real estate delighted present modern 3...,dubai,arabian ranches,1240.808824,False,False,7.0,"[0.0988273, -0.0953468, -0.032153606, 0.045324...",0.078962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666,Brand New | Renewable Energy | Modern Design,"the sustainable city - yas island, yas island,...",5.0,3.0,2024-07-22T11:43:13Z,Residential for Sale,3500000,False,sell,2935.000,NO,view community type 3 maid townhouse size 2935...,abu dhabi,the sustainable city,1192.504259,False,False,8.0,"[0.13066787, -0.016243717, -8.579382e-05, 0.03...",0.136525
4667,Best Deal I Lowest Premium I Well Priced,"diva, yas island, abu dhabi",1.0,0.0,2024-07-22T11:43:12Z,Residential for Sale,630000,False,sell,352.000,YES,✅diva new waterfront residential development...,abu dhabi,yas island,1789.772727,False,False,1.0,"[0.10905955, -0.06309053, 0.037915666, 0.01741...",0.180496
4672,INVESTOR DEAL | PAYMENT PLAN | MOTIVATED SELLER,"elora, the valley, dubai",4.0,3.0,2024-08-01T14:08:07Z,Residential for Sale,2040000,False,sell,2094.000,NO,townhouse sale elora valley primedale real ...,dubai,the valley,974.212034,False,False,7.0,"[0.122611605, -0.08320517, 0.02265256, 0.01044...",0.171419
4677,High ROI | Burj Khalifa View | High Floor,"sobha one tower d, sobha hartland, mohammed bi...",1.0,1.0,2024-06-25T10:56:13Z,Residential for Sale,1339000,True,sell,550.000,NO,sobha one spreads tapestry five interlinked ...,dubai,mohammed bin rashid city,2434.545455,False,False,2.0,"[0.107071616, -0.050193083, -0.050642915, 0.12...",0.168965


In [41]:
df.loc[df["description"].str.contains(fr"{core_building_amenities[0]}")]

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,Emirate,location,price_per_sqft,has_plus_bedroom,has_plus_bathroom,Total_rooms,Embeddings,similarity
1,Full Sea View | Beach Life | Brand New Residence,"la vie, jumeirah beach residence, dubai",3.0,2.0,2024-08-13T05:45:16Z,Residential for Sale,7200000,True,sell,1410.0,YES,apartment sale la vie jumeirah beach residen...,dubai,jumeirah,5106.382979,False,False,5.0,"[0.0934509, -0.047142066, 0.04818736, 0.011824...",0.192513
6,High Floor | Motivated Seller | Full Marina View,"vida residences dubai marina, dubai marina, dubai",3.0,2.0,2024-08-16T05:03:07Z,Residential for Sale,4980000,True,sell,1095.0,NO,property features 2 bedrooms unfurnished high ...,dubai,dubai marina,4547.945205,False,False,5.0,"[0.05768374, -0.017169476, 0.022535445, 0.0257...",0.154097
7,Genuine Resale | Single Row | Payment Plan | E...,"anya 2, arabian ranches 3, dubai",4.0,3.0,2024-08-06T12:33:43Z,Residential for Sale,2800000,True,sell,1991.0,NO,afs properties proudly presents 3 bedroom vil...,dubai,arabian ranches,1406.328478,False,False,7.0,"[0.09001262, -0.070260815, -0.011914495, 0.052...",0.174785
15,Vacant | Fully Furnished | Brand New,"bloom towers b, bloom towers, jumeirah village...",1.0,1.0,2024-08-13T06:11:46Z,Residential for Sale,975000,True,sell,654.0,YES,azco real estate delighted present lavish 1...,dubai,jumeirah,1490.825688,False,False,2.0,"[0.07106599, -0.021113273, -0.050926406, 0.039...",0.095977
16,No Agent | Single Row | View Today,"parkside 3, emaar south, dubai south (dubai wo...",5.0,4.0,2024-08-17T11:10:09Z,Residential for Sale,2950000,True,sell,2555.0,NO,detroit real estate excited present stunnin...,dubai,dubai south,1154.598826,False,False,9.0,"[0.10946861, -0.054709025, -0.030621005, 0.005...",0.248435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689,UPGRADED | VASTU | MARBLE FLOORING,"harmony, tilal al ghaf, dubai",5.0,4.0,2024-07-12T06:38:04Z,Residential for Sale,7600000,True,sell,4505.0,NO,avcor real estate pleased present four bedr...,dubai,tilal al ghaf,1687.014428,False,False,9.0,"[0.094343156, -0.07385898, -0.017212965, 0.028...",0.105879
4690,Mina Al Arab - Gated Community - Stunning Loca...,"lagoon b14, the lagoons, mina al arab, ras al ...",2.0,1.0,2024-07-11T12:53:42Z,Residential for Sale,570000,False,sell,628.0,NO,propertylagoon building 14 – one bedroom apar...,ras al khaimah,mina al arab,907.643312,False,False,3.0,"[0.06988058, -0.05046051, 0.020477576, 0.08374...",0.100915
4692,Emaar | Waterfront Apartment | 10% DP | 5-Yr PP,"al marjan island, ras al khaimah",2.0,1.0,2024-08-12T13:08:33Z,Residential for Sale,3007888,False,sell,854.0,NO,discover address residence emaar nestled pi...,ras al khaimah,al marjan island,3522.117096,False,False,3.0,"[0.043114364, 0.011016979, -0.024703698, 0.020...",0.239151
4693,Upgraded | Spacious With Private Garden,"oia residence, motor city, dubai",5.0,4.0,2024-04-20T08:43:16Z,Residential for Sale,4000000,True,sell,3813.0,NO,rocky real estate presents exclusive upgrade...,dubai,motor city,1049.042748,False,False,9.0,"[0.061551835, -0.08241684, -0.0026803408, 0.03...",0.034315


In [35]:
core_building_amenities[0]

'swimming pool'